In [ ]:
import geopandas as gpd
import momepy as mm
from tqdm import tqdm
from shapely.wkt import loads
import numpy as np
from scipy.spatial import Voronoi
import pandas as pd
from shapely.geometry import Point, Polygon, MultiPoint
from osgeo import ogr
import shapely

import time

# input
network = gpd.read_file(
    'files/AMS/tempnet.gpkg', layer='tempnet'
)
# network = network.loc[network.type.isin(['LineString', 'MultiLineString'])]
# network_cleaned = mm.network_false_nodes(network_cl)
# network_cleaned['uID'] = range(len(network_cleaned))
# network = network_cleaned
# network = network.loc[~network.NFC.isin(['Interstate', 'Other Freeway'])]
buildings = gpd.read_file('files/AMS/elements.gpkg', layer='buildings')

network.crs == buildings.crs

start = time.time()
# parameters
max_size = 100000
size = 5000
circom_min = 0.2
circom_max = 0.75
selfsnap_tolerance = 5
bowtie_tolerance = 0.4


#########
def _remove_leading_out(input, context, tolerance=0.25):
    """
    context defines if it leads somewhere

    TODO: use rtree to speed up intersections
    """
    to_remove = []
    for i, ls in input.iteritems():
        first = Point(ls.coords[0]).buffer(tolerance)
        last = Point(ls.coords[-1]).buffer(tolerance)
        remove = False
        for p in [first, last]:
            if not input.drop(i).intersects(p).any() and not context.intersects(p).any():
                remove = True
        if remove:
            to_remove.append(i)
    return input.drop(to_remove)


def _snap(input, target, tolerance, min=True):
    """
    min True snaps to closest within tolerance, False to all within tolerance

    TODO: use rtree to get distances only to relevant geoms
    """
    input = input.copy()
    for i, geom in input.iteritems():
        distances = target.distance(geom)
        if min:
            close_geom = target.loc[distances.idxmin()]
            geom = shapely.ops.snap(geom, close_geom, tolerance)
        else:
            close_geom = target.loc[distances < tolerance]
            for p in close_geom:
                geom = shapely.ops.snap(geom, p, tolerance)

        input.loc[i] = geom

    return input


def _split(input, target, tolerance):
    input = input.copy()
    buf = target.buffer(tolerance)
    for i, geom in input.iteritems():
        intersects = target.loc[buf.intersects(geom)]
        if not intersects.empty:
            geom = shapely.ops.split(geom, intersects.unary_union)

            input.loc[i] = geom

    return input


def _drop_duplicates(geoms):
    mp = {}
    for ix, geom in geoms.iteritems():
        inters = geoms.intersection(geom).type == 'MultiPoint'

        if inters.any():
            mp[ix] = inters.loc[inters].index.to_list()[0]

    drop = []
    keep = []
    for k in mp.keys():
        if k not in keep:
            drop.append(k)
            keep.append(mp[k])

    geoms = geoms.drop(drop)
    return geoms


def _get_snapping_centroids(input, tolerance):
    """
    generate centroids from close-by points
    """
    points = []
    for i, ls in input.iteritems():
        first = Point(ls.coords[0])
        last = Point(ls.coords[-1])
        for p in [first, last]:
            if input.drop(i).intersects(p.buffer(tolerance)).any():
                points.append(p)
    points = gpd.GeoSeries(points)
    points = points.buffer(tolerance / 2)
    union = points.unary_union
    exploded = gpd.GeoSeries([union]).explode()
    return exploded.centroid


# polygonize network
polygonized = shapely.ops.polygonize(network.geometry)
geoms = [g for g in polygonized]
gdf = gpd.GeoDataFrame(geometry=geoms, crs=network.crs)


# calculate parameters
gdf["area"] = gdf.geometry.area
gdf["circom"] = mm.CircularCompactness(gdf, "area").series


# select valid and invalid network-net_blocks
possible = gdf.loc[gdf["area"] < max_size]
possible = possible.loc[
    (possible["circom"] > circom_max)
    | (possible["circom"] < circom_min)
    | (possible["area"] < size)
]


# check for buildings
buildings["geometry"] = buildings.geometry.representative_point()
sindex = buildings.sindex

drop = []
for index, row in tqdm(possible.iterrows()):
    possible_matches_index = list(sindex.intersection(row.geometry.bounds))
    possible_matches = buildings.iloc[possible_matches_index]
    if possible_matches.intersects(row.geometry).any():
        drop.append(index)

invalid = possible.drop(drop)


# INSERTED - TO BE REMOVED
invalid_manual = gpd.read_file('files/AMS/tempnet.gpkg', layer='invalid', )
pts = invalid_manual.geometry.representative_point()
trueinv = []
for i, r in invalid.iterrows():
    if pts.intersects(r.geometry).any():
        trueinv.append(i)
invalid = gdf[gdf.index.isin(trueinv)].copy()
# UNTIL HERE

valid = gdf[~gdf.index.isin(invalid.index)].copy()


# select edges of valid as an input for tessellation
# valid / network
input = []
sidx = network.sindex
union = invalid.geometry.unary_union
unioned = gpd.GeoSeries(union).explode().reset_index(drop=True)
for i, r in tqdm(unioned.iteritems()):
    possible_matches_index = list(sidx.intersection(r.bounds))
    possible_matches = network.iloc[possible_matches_index]
    real = network.intersection(r.exterior)
    for ix, geom in real.geometry.iteritems():
        input.append(geom)
inter = gpd.GeoSeries(input)
inters = inter[~(inter.is_empty | inter.isna())]
geom_types = inters.type
line_idx = np.asarray(
    (geom_types == "LineString")
    | (geom_types == "MultiLineString")
    | (geom_types == "LinearRing")
)
intersections = inters[line_idx]


# densify interesections ahead of tessellation
def _densify(geom, segment):
    """
    Returns densified geoemtry with segments no longer than `segment`.
    """
    poly = geom
    wkt = geom.wkt  # shapely Polygon to wkt
    geom = ogr.CreateGeometryFromWkt(wkt)  # create ogr geometry
    geom.Segmentize(segment)  # densify geometry by 2 metres
    geom.CloseRings()  # fix for GDAL 2.4.1 bug
    wkt2 = geom.ExportToWkt()  # ogr geometry to wkt
    try:
        new = loads(wkt2)  # wkt to shapely Polygon
        return new
    except Exception:
        return poly


dense = intersections.geometry.apply(_densify, segment=2)

# generate point array for tessellation
points = []
ids = []
for ix, r in dense.items():
    if r.type == "MultiLineString":
        for line in r:
            point_coords = line.coords
            row_array = np.array(point_coords[1:-1] if len(point_coords) > 2 else point_coords).tolist()
            for i, a in enumerate(row_array):
                points.append(row_array[i])
                ids.append(ix)
    elif r.type == "LineString":
        point_coords = r.coords
        row_array = np.array(point_coords[1:-1] if len(point_coords) > 2 else point_coords).tolist()
        for i, a in enumerate(row_array):
            points.append(row_array[i])
            ids.append(ix)


# generate tessellation
voronoi_diagram = Voronoi(np.array(points))


# generate regions
def _regions(voronoi_diagram, unique_id, ids, crs):
    """
    Generate GeoDataFrame of Voronoi regions from scipy.spatial.Voronoi.
    """
    # generate DataFrame of results
    regions = pd.DataFrame()
    regions[unique_id] = ids  # add unique id
    regions["region"] = voronoi_diagram.point_region  # add region id for each point

    # add vertices of each polygon
    vertices = []
    for region in regions.region:
        vertices.append(voronoi_diagram.regions[region])
    regions["vertices"] = vertices

    # convert vertices to Polygons
    polygons = []
    for region in tqdm(regions.vertices, desc="Vertices to Polygons"):
        if -1 not in region:
            polygons.append(Polygon(voronoi_diagram.vertices[region]))
        else:
            polygons.append(None)
    # save polygons as geometry column
    regions["geometry"] = polygons

    # generate GeoDataFrame
    regions_gdf = gpd.GeoDataFrame(regions.dropna(), geometry="geometry")
    regions_gdf = regions_gdf.loc[
        regions_gdf["geometry"].length < 1000000
    ]  # delete errors
    regions_gdf = regions_gdf.loc[
        regions_gdf[unique_id] != -1
    ]  # delete hull-based cells
    regions_gdf.crs = crs
    return regions_gdf


regions_gdf = _regions(voronoi_diagram, "uID", ids, crs=network.crs)
tessellation = regions_gdf[["uID", "geometry"]].dissolve(by="uID", as_index=False)


# make linestrings
linestrings = tessellation.geometry.exterior

# clip linestrings
# use geopandas.clip once released
clipped = linestrings.intersection(invalid.unary_union)
clipped = clipped[~clipped.is_empty & clipped.notnull()]

clipped = clipped.reset_index(drop=True).explode().reset_index(drop=True)

# split at corners
sindex = clipped.sindex
for ix, geom in tqdm(clipped.iteritems(), total=clipped.shape[0]):
    corners = []
    coords = geom.coords
    for i in coords:
        point = Point(i)
        possible_matches_index = list(sindex.intersection(point.bounds))
        possible_matches = clipped.iloc[possible_matches_index]
        precise_matches = sum(possible_matches.intersects(point))
        if precise_matches > 2:
            corners.append(point)
    if len(corners) > 1:
        corners = MultiPoint(corners)
        clipped.loc[ix] = shapely.ops.split(geom, corners)
    elif len(corners) == 1:
        clipped.loc[ix] = shapely.ops.split(geom, corners[0])
clipped = clipped.explode().reset_index(drop=True)


# check duplicates shapely.ops.shared_paths
unique = []
for ix, line in tqdm(clipped.iteritems(), total=clipped.shape[0]):
    if not any(l.equals(line) for l in unique):
        unique.append(line)

unique = gpd.GeoSeries(unique, crs=network.crs)


# simplify
unique = unique.simplify(0.5, preserve_topology=False)  # OK

# fix bowties - snap
# get snapping centroids
centroids = _get_snapping_centroids(unique, bowtie_tolerance)

# snap to centroids
# no_false = _snap(no_false, centroids, bowtie_tolerance, min=True)
no_false = _snap(unique, centroids, bowtie_tolerance, min=True)  # OK


# remove unwanted lines from network. GeoPandas PR is used
overlay = gpd.overlay(network, invalid, keep_geom_type=False, how="difference")  # OK


# remove those leading nowhere
buffered = overlay.geometry.buffer(4)

cleaned = _remove_leading_out(no_false, buffered)  # OK 1
cleaned = mm.network_false_nodes(cleaned, precision=3)  # OK 2
cleaned = cleaned.simplify(0.5, preserve_topology=False)  # 25

# fix unprecise intersections (collapse) - snap
# get snapping centroids
centroids = _get_snapping_centroids(cleaned, selfsnap_tolerance)  # c1

# snap to centroids
cleaned = _snap(cleaned, centroids, selfsnap_tolerance, min=False)  # 3
cleaned = _split(cleaned, centroids, selfsnap_tolerance).explode()  # 4 OK


# remove those leading nowhere again = result fo snap
cleaned = _remove_leading_out(cleaned, buffered, tolerance=0.01)  # 5  OK
cleaned.reset_index(drop=True, inplace=True)

cleaned = mm.network_false_nodes(cleaned, precision=3)  # 6 OK

cleaned = _drop_duplicates(cleaned)  # 7 OK


# snap to network
cleaned = _snap(cleaned, overlay.geometry, selfsnap_tolerance, min=True)  # 8 OK
cleaned = _remove_leading_out(cleaned, overlay, tolerance=0.000001)  # 9 OK

# combine together
combined = cleaned.geometry.append(overlay.geometry).reset_index(drop=True)

# merge together
final = mm.network_false_nodes(combined, precision=3)
print(time.time() - start, 'seconds')
# DONE

final.to_file('files/AMS/tempnet.gpkg', driver='GPKG', layer='network_simplified')